<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/Day_Trading_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pyfolio
import pyfolio as pyf

**DAY TRADING STRATEGY**

# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

In [ ]:
# For data manupulations

START = '2022-08-01'
END = '2023-08-01'

VOLATILITY_FACTOR = 0.45
LOOKBACK = 45

# VF    LB    PEAK_CUM  CUM_LAST
# 0.25  45    3.83       2.54
# 0.35  45    4.46       2.84
# 0.40  45    4.44       2.84
# 0.45  45    4.446      3.095   ******* best
# 0.45  55    3.91       2.5
# 0.45  25    2.51       1.66
# 0.45  65    4.05       2.59
# 0.45  90    4.3        2.7
# 0.50  45    4.34       3.02
# 0.55  45    3.84       2.67
# 1.00  45    3.76       2.62

import pandas as pd
import numpy as np
import yfinance as yf

# For plotting
import matplotlib.pyplot as plt

Ticker = "CANTE"
# Read data from Yahoo finance
TickerIS = Ticker + ".IS"
data = yf.download(TickerIS,START, END)

data_close = data['Adj Close']
# Convert index into datetime format
data.index = pd.to_datetime(data.index,format="%d-%m-%Y")

# Calculate adjustment factor
adjusted_factor = data['Adj Close'] / data['Close']

# Calculate adjusted open price
data['Adj Open'] = adjusted_factor * data['Open']

data.tail()

In [ ]:
# Calculate returns
data['returns'] = (data['Adj Open'] - data['Adj Close'].shift(1)) / data['Adj Close'].shift(1)

In [ ]:
# Calculate standard deviation of returns
data['std'] = data['returns'].rolling(LOOKBACK).std()

In [ ]:
data['positions'] = np.nan

In [ ]:
# GAP UP-GAP DOWN STRATEGY

# Long entry condition
# long_entry = data['Adj Open'] > data['Adj Close'].shift(1)

# # Short entry condition
# short_entry = data['Adj Open'] < data['Adj Close'].shift(1)

In [ ]:
# # IMPROVED STRATEGY

# Long entry condition
longs_entry = data['returns'] < VOLATILITY_FACTOR * data['std']

# Short entry condition
short_entry = data['returns'] > VOLATILITY_FACTOR * data['std']


In [ ]:
# Store 1 when long entry condition is true
data.loc[long_entry, 'positions'] = 1

# Store -1 when short entry condition is true
# data.loc[short_entry, 'positions'] = -1

# Store 0 when wait condition is true
data.loc[short_entry, 'positions'] = 0

data['positions'].fillna(method='ffill', inplace=True)

# Drop NaN values
#data = data.dropna()

In [ ]:
data['strategy_returns'] = ((data['Adj Close'] - data['Adj Open']) / data['Adj Open']) * data.positions

In [ ]:
plt.subplot(3, 1, 1)
plt.plot(data_close)
plt.title(f"{Ticker} time series")

data_log_returns = data_close.pct_change().apply(lambda x: np.log(1+x))

plt.subplot(3, 1, 2)
plt.plot(data_log_returns)
plt.title(f"{Ticker} returns time series")

plt.subplot(3, 1, 3)
data_vlt = data_close.rolling(LOOKBACK).std()*(252**0.5)
plt.title(f"{Ticker} volatility time series")
plt.plot(data_vlt)

fig = plt.gcf()
fig.set_size_inches(16, 11)
plt.show()

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(14,11))

plt.subplot(3, 1, 1)
plt.plot(data_close)
plt.title(f"{Ticker} time series", fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Price', fontsize=14)
plt.grid()

plt.subplot(3, 1, 2)
data['cumulative_strategy_returns'] = (data.strategy_returns+1).cumprod()
data['cumulative_strategy_returns'].fillna(method='ffill', inplace=True)

plt.plot(data.cumulative_strategy_returns)
plt.title(f"{Ticker} returns time series")
plt.grid()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Cumulative strategy returns
# Define the label for the title of the figure
plt.title(f"{Ticker} Cumulative Strategy Returns", fontsize=16)
# Define the labels for x-axis and y-axis
plt.xlabel('Date', fontsize=14)
plt.ylabel('Cumulative Returns', fontsize=14)

# Define the tick size for x-axis and y-axis
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


plt.subplot(3, 1, 3)
plt.xlabel('Date', fontsize=14)
plt.ylabel(f'{Ticker} volatility', fontsize=14)
plt.grid()
plt.xticks(fontsize=12)
plt.title(f"{Ticker} volatility")
plt.plot(data_vlt)


plt.show()

In [ ]:
data.tail(59)

In [ ]:
pyf.create_simple_tear_sheet(data.strategy_returns)

In [ ]:
np.max(data.cumulative_strategy_returns)

In [ ]:
(data.cumulative_strategy_returns[-1])